In [5]:
print('hello')

hello


In [2]:
#from langchain import PrompTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PC
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import langchain
print(langchain.__version__)
#from langchain_pinecone import PineconeVectorStore

0.1.16


In [2]:
PINECONE_API_KEY = "ed6b1ca9-cbb9-41b7-b8b6-86f044793754"
PINECONE_API_ENV = 'gcp-starter'


In [3]:
# Extract
def load_pdf(data):
    loader = DirectoryLoader(data,glob='*.pdf',loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [4]:
extracted_data = load_pdf('data/')

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)

In [4]:
def download_hf_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hf_embeddings()

In [12]:
query_result = embeddings.embed_query('Hello!')
print(query_result,len(query_result))

[-0.06062675267457962, 0.039964884519577026, 0.032226212322711945, 0.048891451209783554, -0.05963549017906189, -0.03785908967256546, 0.028251444920897484, 0.004900546744465828, -0.11331260204315186, -0.02029983513057232, 3.422413283260539e-05, 0.01789705827832222, 0.018761683255434036, -0.052426811307668686, 0.029521584510803223, -0.018175441771745682, 0.00871913693845272, -0.02320040576159954, -0.06915299594402313, 0.02742273546755314, -0.014862298034131527, 0.018731724470853806, -0.0228003840893507, 0.051175545901060104, -0.06328506022691727, -0.04479140788316727, 0.01012916024774313, 0.03268591687083244, -0.005297136027365923, -0.09583697468042374, 0.04735589772462845, 0.050354331731796265, 0.04973457008600235, 0.008949795737862587, 0.0032749499659985304, 0.07771863043308258, -0.09297500550746918, -0.09170224517583847, 0.0582631379365921, 0.00990992784500122, -0.038045190274715424, -0.03432802855968475, -0.007856595329940319, -0.02784900739789009, 0.03519044816493988, -0.01262140925

In [16]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
from langchain_pinecone import PineconeVectorStore

index_name = "medical-chatbot"
index = pc.Index('medical-chatbot')

os.environ['PINECONE_API_KEY'] = 'ed6b1ca9-cbb9-41b7-b8b6-86f044793754'
docsearch = PineconeVectorStore.from_texts(
        [t.page_content for t in text_chunks],
        index_name=index_name,
        embedding=embeddings)

In [5]:
from langchain_pinecone import PineconeVectorStore
import os
index_name = "medical-chatbot"
os.environ['PINECONE_API_KEY'] = 'ed6b1ca9-cbb9-41b7-b8b6-86f044793754'
docsearch = PineconeVectorStore.from_existing_index(index_name,embeddings)

In [21]:
query = 'What are allergies ?'
docs = docsearch.similarity_search(query,k=3)
print(docs)

[Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .\nThe particular allergens to which a person is sensi-'), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts

In [7]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [8]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [13]:

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [14]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])
# https://docs.langflow.org/components/chains

Response :  It depends on the severity of the acne. For mild to moderately severe acne, lotions, soaps, gels, and creams containing benzoyl peroxide or tretinoin may be used to clear up the acne. Isotretinoin (Accutane) is prescribed only for very severe, disfiguring acne.
